In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import *
import time

%matplotlib inline

In [52]:
jailbooking = pd.read_csv("booking.csv")
entries = pd.read_csv("entries_201805311548.csv")
mentalhealth = pd.read_csv('health.csv')
inmate = pd.read_csv('jocojims2inmatedata_201805311745.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,10,14,18,19,26,30,31,32,34,35,36,37,38,40,49,52,64,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
inmate.columns

Index(['inmate_no', 'dob', 'age', 'original_sex', 'sex', 'original_race',
       'race', 'hair', 'eyes', 'height', 'weight', 'mar_stat', 'city', 'state',
       'zip', 'country', 'birth_place', 'arr_agency', 'original_case_number',
       'case_no', 'arrest_dt', 'arrest_time', 'trans_agency', 'control_',
       'bk_dt', 'book_time', 'bonding', 'arr_location', 'b_state', 'mni_no',
       'location', 'license_state', 'veh_lic__', 'veh_lic_state',
       'veh_lic_year', 'veh_make', 'veh_model', 'veh_color', 'dna_draw',
       'l_e_case_', 'same_as_', 'rel_date', 'release_time',
       'associated_records', 'driver_s_lic_type',
       'new_booked_inmate___orientation', 'exp_stamp', 'complexion', 'cfn',
       'demographic_code', 'pros_wksht__', 'ethnic', 'mnh_refer_dt', 'lgbt',
       'at_risk', 'exploited', 'past_year', 'concerns', 'int_dt',
       'notify_asap', 'notify_verbal', 'religion', 'diet', 'rstr_liv_cond',
       'printed_chow', 'prev_bkno', 'ofndr_reg_req', 'ofndr_stamp',
     

In [4]:
inmate = inmate[['race', 'mar_stat', 'city', 'state', 'zip', 'dedupe_id']]

In [6]:
inmate = inmate.drop_duplicates(subset=['dedupe_id'])

In [8]:
inmate.shape

(144254, 6)

In [11]:
inmate.dedupe_id.nunique()

144253

In [12]:
entries.rename(columns={'foreign_id' : 'mni_no'}, inplace=True)
entries = entries[["mni_no", "dedupe_id", 'dob',  'sex']]

In [15]:
entries.shape

(899470, 4)

In [13]:
ent_inmate = pd.merge(entries, inmate, left_on='dedupe_id', right_on='dedupe_id')

In [14]:
ent_inmate.shape

(366077, 9)

In [23]:
ent_inmate.to_csv("ent_inmate.csv")

In [17]:
jailbooking.columns

Index(['mni_no', 'case_no', 'case_type', 'booking_no', 'arresting_agency',
       'arresting_agency_type', 'booking_date', 'release_date', 'bail_type',
       'bail_amt', 'bailed_out'],
      dtype='object')

In [19]:
jailbooking = jailbooking.dropna(subset=['mni_no'])

In [20]:
jailbooking.shape

(28579, 11)

In [16]:
ent_inmate[ent_inmate['dedupe_id']==487227]

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip
184403,669258,487227.0,1973-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184404,669263,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184405,141326,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184406,669265,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184407,669261,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184408,669260,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184409,669262,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184410,669264,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184411,669259,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103


In [29]:
withbooking = pd.merge(ent_inmate, jailbooking, on='mni_no')

In [38]:
withbooking[withbooking['dedupe_id']==309]

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,case_type,booking_no,arresting_agency,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out
11129,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061,14DV00270,DV,14002590,OLATHE P.D.,CITY,2014-03-04 00:00:00,2014-03-05 00:00:00,PR,1500.0,True
11130,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061,15DV00382,DV,15003546,LENEXA P.D.,CITY,2015-03-26 00:00:00,2015-03-27 00:00:00,SUR,1000.0,True
11131,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061,16DV00234,DV,16002371,LENEXA P.D.,CITY,2016-02-24 00:00:00,2016-04-28 00:00:00,NaN,NaN,False


In [39]:
withbooking.to_csv("fixed_withbooking.csv")

In [110]:
mentalhealth = pd.read_csv('health.csv')

In [111]:
mentalhealth.columns

Index(['hash_ssn', 'hash_ssn4', 'hash_lname', 'hash_fname', 'patid', 'dob',
       'race', 'sex', 'hash_gcstreetaddress', 'city', 'state', 'zip',
       'gcvalidity', 'gcsrctabl', 'gcscore', 'gccity', 'gccounty', 'gcstate',
       'gczipcode', 'hash_gclonglat', 'gctract2010id', 'gcblockgroup2010id',
       'gcblock2010id', 'admit_date', 'dschrg_date', 'program', 'pri_dx_code',
       'pri_dx_value', 'refferal_source', 'case_id', 'dedupe_id'],
      dtype='object')

In [112]:
mentalhealth = mentalhealth[['dedupe_id','pri_dx_value', 'refferal_source']]

In [113]:
mentalhealth.head()

,dedupe_id,pri_dx_value,refferal_source
0,190642.0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN
1,1780.0,DEPRESSIVE DISORDER NOS,NaN
2,583264.0,"BIPOLAR DISORDER, DEPRESSED, SEVERE W/O PSYCHO...",NaN
3,666871.0,"SCHIZOPHRENIC DISORDER, SCHIZO-AFFECTIVE TYPE,...",NaN
4,114636.0,"MANIC DISORDER, SINGLE EPISODE, IN FULL REMISSION",SELF


In [123]:
mentalhealth['refferal_source'].loc[mentalhealth['refferal_source'] != 'SELF'] = 'OTHERS'

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [124]:
mentalhealth.head()

,dedupe_id,pri_dx_value,refferal_source
0,190642.0,UNSPECIFIED AFFECTIVE PSYCHOSIS,OTHERS
1,1780.0,DEPRESSIVE DISORDER NOS,OTHERS
2,583264.0,"BIPOLAR DISORDER, DEPRESSED, SEVERE W/O PSYCHO...",OTHERS
3,666871.0,"SCHIZOPHRENIC DISORDER, SCHIZO-AFFECTIVE TYPE,...",OTHERS
4,114636.0,"MANIC DISORDER, SINGLE EPISODE, IN FULL REMISSION",SELF


In [125]:
mentalhealth['pri_dx_value'].fillna('missing', inplace = True)

In [126]:
mentalhealth['refferal_source'].fillna('OTHERS', inplace = True)

In [144]:
mentalhealth.drop_duplicates(subset=['dedupe_id'])

,dedupe_id,pri_dx_value,refferal_source
0,190642.0,UNSPECIFIED AFFECTIVE PSYCHOSIS,OTHERS
1,1780.0,DEPRESSIVE DISORDER NOS,OTHERS
2,583264.0,"BIPOLAR DISORDER, DEPRESSED, SEVERE W/O PSYCHO...",OTHERS
3,666871.0,"SCHIZOPHRENIC DISORDER, SCHIZO-AFFECTIVE TYPE,...",OTHERS
4,114636.0,"MANIC DISORDER, SINGLE EPISODE, IN FULL REMISSION",SELF
5,38306.0,PANIC DISORDER,OTHERS
6,197461.0,"MAJOR DEPRESSIVE DISORDER, RECURRENT, MODERATE",OTHERS
7,74789.0,ATTENTION DEFICIT DISORDER WITH HYPERACTIVITY,OTHERS
8,439589.0,PROLONGED POST TRAUMATIC STRESS DISORDER,OTHERS
9,716225.0,"SCHIZOPHRENIC DISORDER, PARANOID TYPE",OTHERS


In [145]:
almost_done = pd.merge(withbooking, mentalhealth, on='dedupe_id', how='left')

In [146]:
almost_done.shape

(30103, 21)

In [135]:
withbooking=withbooking.drop_duplicates()

In [136]:
withbooking[withbooking['mni_no']==658145]

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,case_type,booking_no,arresting_agency,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out
3,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021,15CR02335,CR,15011570,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2015-10-07 00:00:00,2015-10-07 00:00:00,SUR,10000.0,True


In [137]:
withbooking.dedupe_id.nunique()

21258

In [143]:
withbooking.shape

(27904, 19)

In [138]:
withbooking.mni_no.nunique()

21130

In [147]:
almost_done

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,booking_no,arresting_agency,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source
0,679537,301831.0,1974-04-02,MALE,WHITE,M,SHAWNEE,KS,66226,15CR00773,...,15003768,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2015-04-01 00:00:00,2015-04-01 00:00:00,CA,10000.0,True,"MAJOR DEPRESSIVE DISORDER, SINGLE EPISODE, MOD...",OTHERS
1,431835,296410.0,1971-01-01,MALE,WHITE,NaN,OLATHE,KS,66062,13CR00129,...,13000836,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2013-01-19 00:00:00,2013-01-22 00:00:00,SUR,10000.0,True,SOCIAL PHOBIA,OTHERS
2,431835,296410.0,1971-01-01,MALE,WHITE,NaN,OLATHE,KS,66062,14CR00932,...,14005131,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2014-05-02 00:00:00,2014-05-02 00:00:00,SUR,5000.0,True,SOCIAL PHOBIA,OTHERS
3,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021,15CR02335,...,15011570,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2015-10-07 00:00:00,2015-10-07 00:00:00,SUR,10000.0,True,NaN,NaN
4,655865,370683.0,1984-08-03,MALE,WHITE,S,OLATHE,KS,66062,13CR00468,...,13002475,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2013-03-03 00:00:00,2013-03-03 00:00:00,SUR,1000.0,True,NaN,NaN
5,631990,467807.0,1975-02-18,MALE,WHITE,M,LEAWOOD,KS,66209,11DV01549,...,11014542,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2011-12-01 00:00:00,2011-12-01 00:00:00,SUR,3500.0,True,NaN,NaN
6,510773,103594.0,1991-06-11,MALE,WHITE,S,OVERLAND PARK,KS,66212,15DV00580,...,15005330,OVERLAND PARK P.D.,CITY,2015-05-07 00:00:00,2015-05-08 00:00:00,PR,2500.0,True,NaN,NaN
7,694772,112221.0,1952-02-01,MALE,WHITE,M,OVERLAND PARK,KS,66210,15CR01391,...,15006763,PRAIRIE VILLAGE P.D.,CITY,2015-06-11 00:00:00,2015-06-12 00:00:00,SUR,5000.0,True,NaN,NaN
8,694772,112221.0,1952-02-01,MALE,WHITE,M,OVERLAND PARK,KS,66210,15CR02428,...,15011602,OVERLAND PARK P.D.,CITY,2015-10-08 00:00:00,2015-10-08 00:00:00,SUR,25000.0,True,NaN,NaN
9,704863,221941.0,1990-04-30,MALE,WHITE,S,KANSAS CITY,MO,64155,16CR00429,...,16001911,KANSAS HIGHWAY PATROL,STATE,2016-02-14 00:00:00,2016-02-14 00:00:00,SUR,2500.0,True,NaN,NaN


In [149]:
almost_done['mh_treatment'] = pd.notna(almost_done["refferal_source"])

In [151]:
almost_done.shape

(30103, 22)

In [153]:
almost_done = almost_done.drop_duplicates()

In [161]:
almost_done = almost_done.dropna(subset = ['dedupe_id'])

In [162]:
almost_done.shape

(29504, 22)

In [155]:
almost_done.columns

Index(['mni_no', 'dedupe_id', 'dob', 'sex', 'race', 'mar_stat', 'city',
       'state', 'zip', 'case_no', 'case_type', 'booking_no',
       'arresting_agency', 'arresting_agency_type', 'booking_date',
       'release_date', 'bail_type', 'bail_amt', 'bailed_out', 'pri_dx_value',
       'refferal_source', 'mh_treatment'],
      dtype='object')

In [157]:
def convert_datetime(df, cols):
    for col in cols:
        df[col] = pd.to_datetime(df[col])

In [158]:
convert_datetime(almost_done, ['booking_date', 'release_date', 'dob'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [159]:
#3 Pre-Process & Filling Missing Data#3 Pre-P 
def Detect_missing_value(df):
    '''
    Find out the columns have missing values

    Input:
        df: (pandas dataframe)
 
    Returns:
        a list of those column names
    '''
    rv = []
    for col in df.columns:
        if df[col].count() < df.shape[0]:
            rv.append(col)
            print(col, "has missing values.")
    return rv

In [163]:
Detect_missing_value(almost_done)

sex has missing values.
mar_stat has missing values.
city has missing values.
state has missing values.
zip has missing values.
arresting_agency has missing values.
arresting_agency_type has missing values.
release_date has missing values.
bail_type has missing values.
bail_amt has missing values.
pri_dx_value has missing values.
refferal_source has missing values.


['sex',
 'mar_stat',
 'city',
 'state',
 'zip',
 'arresting_agency',
 'arresting_agency_type',
 'release_date',
 'bail_type',
 'bail_amt',
 'pri_dx_value',
 'refferal_source']

In [166]:
# get numeric columns 
num_cols = almost_done._get_numeric_data().columns
num_cols

Index(['mni_no', 'dedupe_id', 'booking_no', 'bail_amt', 'bailed_out',
       'mh_treatment'],
      dtype='object')

In [167]:
#and then find out categorical columns
cate_cols = list(set(almost_done.columns) - set(num_cols))
cate_cols

['arresting_agency_type',
 'case_no',
 'mar_stat',
 'zip',
 'sex',
 'dob',
 'bail_type',
 'refferal_source',
 'case_type',
 'race',
 'pri_dx_value',
 'arresting_agency',
 'release_date',
 'city',
 'booking_date',
 'state']

In [168]:
def Fill_in(df, cols, method="mean"):
    '''
    Filling in missing values with "mean" or "median"
    
    Inputs:
        df: (pandas dataframe)
        a list of those column names
        method (string): mean or median
 
    Returns:
        pandas dataframe
    '''
    for col in cols:
        if method =='mean':
            val = df[col].mean()
        elif method =='median':
            val = df[col].median()
        elif method =='mode':
            val = df[col].mode()
            val = str(val)
        elif method == 'missing':
            val = "missing"
        df[col] = df[col].fillna(val)
        print ('Filling missing value for {} using {}'.format(col, method))
    return df

In [169]:
almost_done = Fill_in(almost_done, ['city', 'state', 'race', 'mar_stat'], "missing")

Filling missing value for city using missing
Filling missing value for state using missing
Filling missing value for race using missing
Filling missing value for mar_stat using missing


In [170]:
fill_with_mode = ['sex', 'arresting_agency', 'arresting_agency_type', 'bail_type'] 

In [171]:
almost_done = Fill_in(almost_done, fill_with_mode, "mode")
almost_done = Fill_in(almost_done, ['bail_amt'], "median")

Filling missing value for sex using mode
Filling missing value for arresting_agency using mode
Filling missing value for arresting_agency_type using mode
Filling missing value for bail_type using mode
Filling missing value for bail_amt using median


In [172]:
Detect_missing_value(almost_done)

zip has missing values.
release_date has missing values.
pri_dx_value has missing values.
refferal_source has missing values.


['zip', 'release_date', 'pri_dx_value', 'refferal_source']

In [174]:
almost_done['bailed_out'].head()

0    True
1    True
2    True
3    True
4    True
Name: bailed_out, dtype: bool

In [175]:
#convert dummy var
almost_done['bailed_out'] = almost_done['bailed_out']*1

In [176]:
#convert dummy var
almost_done['mh_treatment'] = almost_done['mh_treatment']*1

In [178]:
almost_done[almost_done['dedupe_id']==296410]

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,arresting_agency,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment
1,431835,296410.0,1971-01-01,MALE,WHITE,missing,OLATHE,KS,66062,13CR00129,...,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2013-01-19,2013-01-22,SUR,10000.0,1,SOCIAL PHOBIA,OTHERS,1
2,431835,296410.0,1971-01-01,MALE,WHITE,missing,OLATHE,KS,66062,14CR00932,...,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2014-05-02,2014-05-02,SUR,5000.0,1,SOCIAL PHOBIA,OTHERS,1


In [179]:
almost_done.to_csv('total.csv', index=False)

In [ ]:
##################up to now

In [ ]:
#2 explore
def create_graph(df, x, y, method='mean', graph_type='line'):
    '''
    Generate distribution graph for specific variable
    
    Input:
        df: (pandas dataframe)
        x(string): the variable/attribute you want to explore
        graph_typee(string): the type of graph you want to draw: line, bar, scatter
        method (string): mean, median or sum
        
    Return:
        a graph for a selected attribute. 
    '''
    cols = [x, y]
    if method == 'mean':
        var = df[cols].groupby(x).mean()
    elif method == 'median':
        var = df[cols].groupby(x).median()
    elif method == 'sum':
        var = df[cols].goupby(x).sum()
    graph = var.plot(kind=graph_type, use_index=False, figsize=(8,4))


In [ ]:
mni_df = entries.groupby(['dedupe_id']).size().reset_index(name='re-entry')

In [ ]:
mni_df.tail()

In [ ]:
mni_df.head()

In [ ]:
df['re-entry'] = np.where(mni_df['re-entry']>1, 1, 0)

In [ ]:
mni_df.head()

In [ ]:
jail = jailbooking.merge(mni_df, how="left")

In [ ]:
jail.tail()

In [ ]:
person.drop(['tract2010id', 'blockgroup2010id', 'block2010id'], inplace=True, axis=1)

In [ ]:
person.columns

In [ ]:
jail_ppl = jail.merge(person, how="left", on="mni_no")

In [ ]:
jail_ppl.shape

In [ ]:
mentalhealth = pd.read_csv("mentalhealth.csv")

In [ ]:
mentalhealth.columns

In [ ]:
mentalhealth.shape

In [ ]:
mentalhealth.drop(['dob','city', 'race', 'sex', 'state', 'zip', 'tract2010id',
       'blockgroup2010id', 'block2010id'], inplace=True, axis=1)

In [ ]:
jail_ppl.head()

In [ ]:
jail_ppl['dob'] = pd.to_datetime(jail_ppl['dob'])
for i, row in jail_ppl.iterrows():
    jail_ppl['age']= pd.to_datetime('2018-06-01').year - jail_ppl['dob'].dt.year

In [ ]:
df.to_csv('df.csv', index=False)
jail_ppl['booking_days'] = jail_ppl['release_date'] - jail_ppl['booking_date']

In [ ]:
#Convert the days type to integer 
jail_ppl['booking_days'] = jail_ppl['booking_days'].astype('timedelta64[D]')

In [ ]:
jail_ppl.drop(['booking_date','release_date'], inplace=True, axis=1)

In [ ]:
jail_ppl.head()

In [ ]:
mentalhealth = mentalhealth.dropna(subset = ['personid'])

In [ ]:
mentalhealth.head()

In [ ]:
mentalhealth['dschrg_date'].fillna('2018-01-01 00:00:00', inplace = True)

In [ ]:
#Convert column type from string to datetime
mentalhealth['admit_date'] = pd.to_datetime(mentalhealth['admit_date'])
mentalhealth['dschrg_date'] = pd.to_datetime(mentalhealth['dschrg_date'], errors='ignore')
mentalhealth['treatment_days'] = mentalhealth['dschrg_date'] - mentalhealth['admit_date']

In [ ]:
#Convert the days type to integer 
mentalhealth['treatment_days'] = mentalhealth['treatment_days'].astype('timedelta64[D]')

In [ ]:
mentalhealth.treatment_days.head()

In [ ]:
def  checking_for_nulls(dataframe):
    '''
    Given a dataframe, checks for columns which have NaN or Nulls,
        and returns a list with the name of those features which have NaN or Nulls.
        
    Input:
        dataframe
        
    Output:
        features_with_nulls: list of strings
    '''
    features = dataframe.columns
    features_with_nulls = []

    for column in df.columns:    
        if df[column].isnull().sum() > 0:
            features_with_nulls.append(column)
    
    return features_with_nulls

In [ ]:
def fill_in_missing_data (dataframe, features, criteria):
    '''
    Given a dataframe and a criteria (options: mean, median or mode),
        fills in the NaN or Null values in that column for the dataframe
        based on the given criteria
        
    Input:
        dataframe
        criteria: string
    '''
    
    for feature in features:
        if criteria == 'mean': input_value = df[feature].mean() 
        if criteria == 'median': input_value = df[feature].median() 
        if criteria == 'mode': input_value = df[feature].mode()
        
        dataframe[feature] = dataframe[feature].fillna(input_value)

In [ ]:
jail_ppl = jail_ppl.dropna(subset = ['personid'])

In [ ]:
jail_ppl.shape

In [ ]:
health = mentalhealth[["personid", "treatment_days"]]

In [ ]:
health.head()

In [ ]:
df = pd.merge(jail_ppl, health, on="personid", how='left')

In [ ]:
df.shape

In [ ]:
df.personid.nunique()

In [ ]:
df.tail()

In [ ]:
df['treatment_days'].fillna(-1, inplace = True)

In [ ]:
df['mental_health'] = np.where(df['treatment_days']>=0.0, 1, 0)

In [ ]:
df.tail()

In [ ]:
cols_to_fill = checking_for_nulls(df)

In [ ]:
cols_to_fill

In [ ]:
fill_in_missing_data(df, ['bail_amt', 'booking_days'], 'median')

In [ ]:
df['bail_type'].mode()

In [ ]:
df['bail_type'] = df['bail_type'].fillna('SUR')

In [ ]:
checking_for_nulls(df)

In [ ]:
df.head()

In [ ]:
create_graph(df, 'bail_amt', 'sum')

In [ ]:
create_graph(df, 'booking_days', 'sum')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

In [ ]:
def corr_matrix(df):
    '''
    Creates a heatmap that shows the correlations between the different variables in a dataframe.
    
    Input:
        df: a dataframe
        title: name of the correlation_matrix
        
    Return:
        Outputs a heatmatrix showing correlations
    
    
    '''
    f, ax = plt.subplots(figsize=(10, 8))
    corr = df.corr()
    sns.heatmap(corr, 
                xticklabels=corr.columns.values,
                yticklabels=corr.columns.values, 
                mask=np.zeros_like(corr, dtype=np.bool), 
                cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

In [ ]:
corr_matrix(df)

In [ ]:
df.columns

In [ ]:
df = df.drop(['mni_no', 'case_no', 'booking_no', 'dob', 'personid'], axis=1)

In [ ]:
df.to_csv('df.csv', index=False)